In [1]:
import tensorflow as tf

In [2]:
model = tf.keras.models.load_model('./0000001')

In [3]:
type(model)

keras.engine.functional.Functional

In [4]:
from zipfile import ZipFile
import os
import cv2
import numpy as np

# with ZipFile('./drive/MyDrive/Final Year Project/Datasets/testROI.zip', 'r') as zipObj:
#    # Extract a specific file from zip file
#    zipObj.extractall('./')

testfilenames = []
images = []
for filename in sorted(os.listdir("./testROI")):
# for filename in (os.listdir("./testROI")):
    testfilenames.append(filename)
    # Read image
    img = cv2.imread(os.path.join("./testROI/",filename), 0)
    
    # Append image to list
    images.append(img)

palm_arrays = np.array(images)

In [6]:
palm_arrays = palm_arrays.astype('float32') / 255.
palm_arrays = palm_arrays.reshape(-1, 150, 150, 1)
print(palm_arrays.shape)

(15, 150, 150, 1)


In [7]:
tests = model.predict(palm_arrays)

1/1 [==============================] - 0s 334ms/step


In [8]:
testfilenames

['004_1.JPG',
 '004_2.JPG',
 '004_3.JPG',
 '004_4.JPG',
 '004_5.JPG',
 '029_2.JPG',
 '029_3.JPG',
 '029_4.JPG',
 '029_5.JPG',
 '029_6.JPG',
 '056_1.JPG',
 '056_2.JPG',
 '056_3.JPG',
 '056_4.JPG',
 '056_5.JPG']

In [10]:
from sklearn.cluster import KMeans

In [11]:
kmeans = KMeans(n_clusters=3, n_init=3)
kmeans.fit(tests)

KMeans(n_clusters=3, n_init=3)

In [13]:
for i in range(len(testfilenames)):
  print("image: ", testfilenames[i], ", cluster: ", kmeans.labels_[i])

image:  004_1.JPG , cluster:  0
image:  004_2.JPG , cluster:  0
image:  004_3.JPG , cluster:  0
image:  004_4.JPG , cluster:  0
image:  004_5.JPG , cluster:  0
image:  029_2.JPG , cluster:  1
image:  029_3.JPG , cluster:  1
image:  029_4.JPG , cluster:  1
image:  029_5.JPG , cluster:  0
image:  029_6.JPG , cluster:  1
image:  056_1.JPG , cluster:  2
image:  056_2.JPG , cluster:  2
image:  056_3.JPG , cluster:  2
image:  056_4.JPG , cluster:  2
image:  056_5.JPG , cluster:  2


In [46]:
#groups images by whom it belongs to
from collections import defaultdict
filename_groups = defaultdict(list)
for filename in testfilenames:
    prefix = filename.split('_')[0]
    filename_groups[prefix].append(filename)

In [37]:
filename_groups

defaultdict(list,
            {'004': ['004_1.JPG',
              '004_2.JPG',
              '004_3.JPG',
              '004_4.JPG',
              '004_5.JPG'],
             '029': ['029_2.JPG',
              '029_3.JPG',
              '029_4.JPG',
              '029_5.JPG',
              '029_6.JPG'],
             '056': ['056_1.JPG',
              '056_2.JPG',
              '056_3.JPG',
              '056_4.JPG',
              '056_5.JPG']})

In [68]:
#groups images by cluster (by co-pilot)
group_clusters = {}
for prefix,filenames in filename_groups.items():
    clusters = [kmeans.labels_[testfilenames.index(filename)] for filename in filenames]
    group_clusters[prefix] = clusters
group_clusters

{'004': [0, 0, 0, 0, 0], '029': [1, 1, 1, 0, 1], '056': [2, 2, 2, 2, 2]}

In [72]:
#finds the most images belonging to each cluster (by co-pilot)
clusters_max = {}
for prefix, cluster_labels in group_clusters.items():
    most_common_cluster = max(set(cluster_labels), key=cluster_labels.count)
    clusters_max[prefix] = most_common_cluster

In [70]:
clusters_max

{'004': 0, '029': 1, '056': 2}

In [65]:
#evaluates to true if the image belongs to the cluster with the most images of its type
evaluations = []
for filename in testfilenames:
    prefix = filename.split("_")[0]
    index = testfilenames.index(filename)
    evaluation = kmeans.labels_[index] == clusters_max[prefix]
    evaluations.append(evaluation)

In [62]:
#percentage of correct predictions
sum(evaluations) / len(evaluations)

0.9333333333333333